## FIRST EXPERIMENT

В первом эксперименте мы получаем реконструкцию при помощи метода, который был описан в демо PixSfM. Суть данного метода заключается в том, что мы при помощи **hloc** получаем локальные features для каждой картинки, создаем matches между каждой пары картинок, затем по полученным данныи и при помощи различных конфигов получаем SfM и записываем результат в папку.

# Libraries

In [ ]:
%load_ext autoreload
%autoreload 2
import tqdm, tqdm.notebook
tqdm.tqdm = tqdm.notebook.tqdm  # notebook-friendly progress bars

from pathlib import Path
import pycolmap
import sys

sys.path.append("/workspace/pixel-perfect-sfm/")
sys.path.append("/workspace/pixel-perfect-sfm/Hierarchical-Localization")

from hloc import extract_features, match_features, reconstruction, pairs_from_exhaustive, visualization
from hloc.visualization import plot_images, read_image
from hloc.utils.viz_3d import init_figure, plot_points, plot_reconstruction, plot_camera_colmap
from hloc.utils.read_write_model import  write_next_bytes, Point3D, Image, read_images_text, read_points3D_binary,\
        write_points3D_binary, write_images_binary, read_images_binary, read_cameras_text

from pixsfm.util.visualize import init_image, plot_points2D
from pixsfm.refine_hloc import PixSfM
from pixsfm import ostream_redirect

import numpy as np
from matplotlib import pyplot as plt

import open3d as o3d
assert o3d.__version__ == '0.15.2', 'The version 0.15.2 is required!'

# redirect the C++ outputs to notebook cells
cpp_out = ostream_redirect(stderr=True, stdout=True)
cpp_out.__enter__()

# Setup

В **object_name** необходимо задать имя объекта, над которым вы хотите провести эксперимент.

**check_for_calibrated_images** - булевая переменная, по которой мы выбираем какие поз камер использовать (менее точные (True) или точные (False)).

**delete_previous_output** - если True, то удаляет все предыдущие файлы в папке outputs. Использовать супер осторожно.

**has_cache** - если True, то у Вас уже существует файл с feature maps и он сохранен в папке cache_init. Это файл с feature maps Вы получаете только тогда, когда вы уже сделали featuremetric KA или BA для одного из ваших экспериментов.

**show_visualization** - если True, то показывает визуализацию результата эксперимента (3d pointcloud, задектированные keypoints (features) и final reprojections для какого-то изображения).

In [ ]:
object_name = 'dragon'

check_for_calibrated_images = False
delete_previous_output = False

has_cache = True
show_visualization = False

**images_init** - путь к файлу images.txt с известными позами камер (каждая вторая строка пустая)

**calibrated_images_init** - путь к файлу images.txt c известнами позами камер (но менее точные)

**cameras_init** - путь к файлу cameras.txt, где хранятся GT параметры камеры

**images** - путь к папке с изображениями для реконструкции

**outputs** - путь к папке со всеми результатами

**cache_init** - путь к кэш-файлу, его мы получаем во время того, когда делаем KA или BA. В этот файле хранятся featuremaps после  dense feature extraction. В среднем на одну картинку размером 2368х1952 уходит 3 минуты. Этот файл вообще нельзя трогать, поэтому мы копируем его в папку outputs для своего эксперимента и продолжаем работу.

**cache_path** - тот же файл, что cache_init, с которым мы теперь будем работать во время эксперимента.

**sfm_pairs** - файл с названиями пар изображений на каждой строке

**features** - файл с features для каждой картинки, извлеченными при помощи feature_conf

**matches** - файл с matches для каждой пары картинок, извлеченными при помощи matcher_conf

In [ ]:
root = Path('/workspace')

images_init = root / f'datasets/sk3d/dataset/{object_name}/tis_right/rgb/images.txt'
calibrated_images_init = root / 'datasets/sk3d/dataset/calibration/tis_right/rgb/images.txt' # менее точные

cameras_init = root / 'datasets/sk3d/dataset/calibration/tis_right/rgb/cameras.txt'

images = root / f'datasets/sk3d/dataset/{object_name}/tis_right/rgb/undistorted/ambient@best'

outputs = root / f'pixel-perfect-sfm/outputs/{object_name}_exp1/'

if delete_previous_output:
    !rm -rf $outputs 
    
outputs.mkdir(parents=True, exist_ok=True)    

if has_cache:
    cache_init = root / f'pixel-perfect-sfm/outputs/caches/{object_name}/s2dnet_featuremaps_sparse.h5'
    !cp -r $cache_init $outputs
    cache_path = outputs / 's2dnet_featuremaps_sparse.h5'

sfm_pairs = outputs / 'pairs-sfm.txt'
features = outputs / 'features.h5'
matches = outputs / 'matches.h5'

Ниже представлены некоторые виды возможных экспериментов, которые можно провести с различным количеством камер и параметрами для камер. Для каждого эксперимента существует своя булевая переменная, которая позволяет выбрать эксперимент.

1) В папке **raw_single_cam** будет сохранена реконструкия, которая была создана при помощи одной камеры модели PINHOLE. Нет BA и KA, строиться так называемая сырая *реконструкция* при помощи **features**, **matches**, **sfm_pairs**.

2) В папке **raw_dir_auto_cam** будет сохранена реконструкия, которая была создана при помощи нескольких камер модели PINHOLE (то есть для каждого изображения будет использоваться своя камера -> 100 изображений = 100 различных камер). Нет BA и KA, строиться так называемая сырая *реконструкция* при помощи **features**, **matches**, **sfm_pairs**.

3) В папке **exp1_dir_single_cam** будет сохранена реконструкия, которая была создана при помощи одной камеры модели PINHOLE. Используется BA и KA (оба featuremetric). Строиться улучшенная (refined) *реконструкция* при помощи **features**, **matches**, **sfm_pairs**. Параметры камеры и их поза будет сгенерены аппаратом PixSfM.

4) В папке **exp1_dir_auto_cam** будет сохранена реконструкия, которая была создана при помощи нескольких камер модели PINHOLE. Используется BA и KA (оба featuremetric). Строиться улучшенная (refined) *реконструкция* при помощи **features**, **matches**, **sfm_pairs**. Параметры камер и их позы будут сгенерены аппаратом PixSfM.

5) В папке **exp1_dir_single_cam_with_params** будет сохранена реконструкия, которая была создана при помощи одной камеры модели PINHOLE. Используется BA и KA (оба featuremetric). Строиться улучшенная (refined) *реконструкция* при помощи **features**, **matches**, **sfm_pairs**. В данном конфиге мы используем GT параметры известной для нас камеры, при этом не изменяя внутренние параметры камеры, а меняя только экстринсики камеры. 

In [ ]:
run_raw_single = True
run_raw_auto = False
run_ref_single = False
run_ref_auto = False
run_ref_single_with_params = True

if run_raw_single:
    raw_dir_single_cam = outputs / "raw_single_cam"
    raw_dir_single_cam.mkdir(parents=True, exist_ok=True)

if run_raw_auto:    
    raw_dir_auto_cam = outputs / "raw_auto_cam"
    raw_dir_single_cam.mkdir(parents=True, exist_ok=True)

if run_ref_auto:
    exp1_dir_auto_cam = outputs / "ref_exp1_auto_cam"
    exp1_dir_auto_cam.mkdir(parents=True, exist_ok=True)
    
if run_ref_single:
    exp1_dir_single_cam = outputs / "ref_exp1_single_cam"
    exp1_dir_single_cam.mkdir(parents=True, exist_ok=True)

if run_ref_single_with_params:    
    exp1_dir_single_cam_with_params = outputs / "ref_exp1_single_cam_with_init_camera_params"
    exp1_dir_single_cam_with_params.mkdir(parents=True, exist_ok=True)

# 3D mapping and refinement

Здесь описаны возможности для настройки [**extract_features**](https://github.com/cvg/Hierarchical-Localization/blob/91f40bfd765add3b59ba7376f8579d8829f7fa78/hloc/extract_features.py#L21)

Здесь описаны возможности для настройки [**match_features**](https://github.com/cvg/Hierarchical-Localization/blob/91f40bfd765add3b59ba7376f8579d8829f7fa78/hloc/match_features.py#L17)

Здесь описан пайплайн того, как можно использовать свои кастомные [**local features**, **matcher**, **image retrieval**](https://github.com/cvg/Hierarchical-Localization/tree/91f40bfd765add3b59ba7376f8579d8829f7fa78#using-your-own-local-features-or-matcher).


In [ ]:
feature_conf = extract_features.confs['superpoint_aachen']
matcher_conf = match_features.confs['superglue']

Здесь мы проверяем какие изображения мы будем использовать для реконструкции.

In [ ]:
references = [str(p.relative_to(images)) for p in images.iterdir()]
print(len(references), "mapping images")
plot_images([read_image(images / r) for r in references[:4]], dpi=50)

**extract_features** - данная функция получает на вход *feature_conf*, *images* (путь к папке с изображениями), *image_list* (список тех изображений, которые вы хотите использовать для feature exctraction), *feature_path* (путь к файлу, где будет сохранен результат). На выходе получаем файл (**features**) с извлеченными features. Если **features** существует, то пропускается.

**pairs_from_exhaustive** - данная функция получает на вход *sfm_pairs* (путь к файлу, где будет сохранен результат), *image_list* (список тех изображений, при помощи которых вы сделаете exhaustive pairs.) На выходе получаем файл (**sfm_pairs**) с парами изображений.  Если **sfm_pairs** существует, то пропускается.

**match_features** - данная функция получает на вход *matcher_conf*, *sfm_pairs* (путь к файлу, где хранятся пары изображений после exhaustive pairing), *features* (путь к файлу, где хранятся извлеченный features для каждого изображения), *matches* (путь к файлу, где хранятся matches для каждой пары изображения). На выходе получаем файл (**matches**) с matches для каждой пары изображений . Если **match_features** существует, то пропускается.



Ниже клетка может выполняться от получаса до часа (в зависимости от нагруженности).

Features extraction - 1 минута.

Features matching - 35-50 минут.

In [ ]:
!cp -r /workspace/pixel-perfect-sfm/outputs/dragon/features.h5 $outputs
!cp -r /workspace/pixel-perfect-sfm/outputs/dragon/matches.h5 $outputs
!cp -r /workspace/pixel-perfect-sfm/outputs/dragon/pairs-sfm.txt $outputs

In [ ]:
extract_features.main(feature_conf, images, image_list=references, feature_path=features) 
pairs_from_exhaustive.main(sfm_pairs, image_list=references)
match_features.main(matcher_conf, sfm_pairs, features=features, matches=matches); 

В папке **raw_dir_auto_cam** будет сохранена реконструкия, которая была создана при помощи нескольких камер модели PINHOLE (то есть для каждого изображения будет использоваться своя камера -> 100 изображений = 100 различных камер). Нет BA и KA, строиться так называемая сырая *реконструкция* при помощи **features**, **matches**, **sfm_pairs**.

Ниже клетка работает примерно 5-6 минут для 100 картинок.

In [ ]:
if run_raw_auto:
    
    conf = {"KA":{"apply": False}, "BA": {"apply": False}}
    
    opts = dict(camera_model='PINHOLE')
    hloc_args = dict(camera_mode=pycolmap.CameraMode.AUTO,
                     verbose=True,  # включены подробные логи
                     image_options=opts)

    raw_sfm = PixSfM(conf=conf)
    
    raw_auto, _ = raw_sfm.reconstruction(raw_dir_auto_cam, 
                                    images, 
                                    sfm_pairs, 
                                    features, 
                                    matches, 
                                    image_list=references,
                                    **hloc_args)
    print(raw_auto.summary())

    !mkdir -p $raw_dir_auto_cam/hloc/model_txt/
    
    !colmap model_converter \
        --input_path $raw_dir_auto_cam/hloc \
        --output_path $raw_dir_auto_cam/hloc/model_txt/ \
        --output_type TXT

В папке **raw_single_cam** будет сохранена реконструкия, которая была создана при помощи одной камеры модели PINHOLE. Нет BA и KA, строиться так называемая сырая *реконструкция* при помощи **features**, **matches**, **sfm_pairs**.

Ниже клетка работает примерно 5-6 минут для 100 картинок.

In [ ]:
if run_raw_single:
    
    conf = {"KA":{"apply": False},  "BA": {"apply": False}}
    
    opts = dict(camera_model='PINHOLE')
    hloc_args = dict(camera_mode=pycolmap.CameraMode.SINGLE,
                     verbose=True,
                     image_options=opts)
    
    raw_sfm = PixSfM(conf=conf)

    raw_single, _ = raw_sfm.reconstruction(raw_dir_single_cam, 
                                    images, 
                                    sfm_pairs, 
                                    features, 
                                    matches, 
                                    image_list=references,
                                    **hloc_args)
    print(raw_single.summary())

    !mkdir -p $raw_dir_single_cam/hloc/model_txt/

    !colmap model_converter \
        --input_path $raw_dir_single_cam/hloc \
        --output_path $raw_dir_single_cam/hloc/model_txt/ \
        --output_type TXT

В папке **exp1_dir_single_cam** будет сохранена реконструкия, которая была создана при помощи одной камеры модели PINHOLE. Используется BA и KA (оба featuremetric). Строиться улучшенная (refined) *реконструкция* при помощи **features**, **matches**, **sfm_pairs**. Создание СNN features - 5 часов для 100 картинок (3 минуты в среднем на 1 картинку размером 2368х1952).

Ниже клетка работает примерно 8-10 минут для 100 картинок.

In [ ]:
if run_ref_single:
    
    opts = dict(camera_model='PINHOLE')
    hloc_args = dict(
                    camera_mode=pycolmap.CameraMode.SINGLE,
                    verbose=True,
                    image_options=opts)
    
    conf = {
            "dense_features": {
                    "use_cache": True,
            },
             "KA": {
                    "dense_features": {'use_cache': True}, 
                    "split_in_subproblems": True,
                    "max_kps_per_problem": 1000,  
                },

            "BA": { 
                "apply": True
                }
            }
    
    if has_cache == False:
        conf.update({
            "dense_features": {
                "use_cache": True,
                "sparse" : True,
                "dtype" : "half",
                "overwrite_cache": True,
                "load_cache_on_init": False,
                "patch_size": 8,
                "cache_format": "chunked"
            }
            
        })
        
    sfm = PixSfM(conf)
    refined1_single, sfm_outputs = sfm.reconstruction(exp1_dir_single_cam, 
                                          images, 
                                          sfm_pairs, 
                                          features, 
                                          matches, 
                                          image_list=references, 
                                          cache_path=cache_path,
                                          **hloc_args)
    print(refined1_single.summary())
    
    if not has_cache:    
        caches = root / f'pixel-perfect-sfm/outputs/caches/exp1/{object_name}'
        caches.mkdir(parents=True, exist_ok=True)
        cache_path = outputs / 's2dnet_featuremaps_sparse.h5'
        !cp -r $cache_path $caches   
        
    !mkdir -p $exp1_dir_single_cam/hloc/model_txt/

    !colmap model_converter \
        --input_path $exp1_dir_single_cam/hloc \
        --output_path $exp1_dir_single_cam/hloc/model_txt/ \
        --output_type TXT    

В папке **exp1_dir_auto_cam** будет сохранена реконструкия, которая была создана при помощи нескольких камер модели PINHOLE. Используется BA и KA (оба featuremetric). Строиться улучшенная (refined) *реконструкция* при помощи **features**, **matches**, **sfm_pairs**. Создание СNN features - 5 часов для 100 картинок (3 минуты в среднем на 1 картинку размером 2368х1952)

Ниже клетка работает примерно 9-11 минут для 100 картинок.

In [ ]:
if run_ref_auto: 

    conf = {
            "dense_features": {
                    "use_cache": True,
            },
             "KA": {
                    "dense_features": {'use_cache': True}, 
                    "split_in_subproblems": True,
                    "max_kps_per_problem": 1000,  
                },

            "BA": { 
                "apply": True
            }
    }
    
    if has_cache == False:
        conf.update({
            "dense_features": {
                "use_cache": True,
                "sparse" : True,
                "dtype" : "half",
                "overwrite_cache": True,
                "load_cache_on_init": False,
                "patch_size": 8,
                "cache_format": "chunked"
            }
            
        })
    
    opts = dict(camera_model='PINHOLE')

    hloc_args = dict(verbose=True,
                    image_options=opts)
    
    sfm = PixSfM(conf)
    
    if has_cache == False:
        refined1_auto, sfm_outputs = sfm.reconstruction(exp1_dir_auto_cam, 
                                              images, 
                                              sfm_pairs, 
                                              features, 
                                              matches, 
                                              image_list=references,
                                              **hloc_args)
    else:
        refined1_auto, sfm_outputs = sfm.reconstruction(exp1_dir_auto_cam, 
                                              images, 
                                              sfm_pairs, 
                                              features, 
                                              matches, 
                                              image_list=references,
                                              cache_path=cache_path,
                                              **hloc_args)

    print(refined1_auto.summary())
    
    if not has_cache:    
        caches = root / f'pixel-perfect-sfm/outputs/caches/exp1/{object_name}'
        caches.mkdir(parents=True, exist_ok=True)
        cache_path = outputs / 's2dnet_featuremaps_sparse.h5'
        !cp -r $cache_path $caches  

    !mkdir -p $exp1_dir_auto_cam/hloc/model_txt/

    !colmap model_converter \
        --input_path $exp1_dir_auto_cam/hloc \
        --output_path $exp1_dir_auto_cam/hloc/model_txt/ \
        --output_type TXT

В папке **exp1_dir_single_cam_with_params** будет сохранена реконструкия, которая была создана при помощи одной камеры модели PINHOLE. Используется BA и KA (оба featuremetric). Строиться улучшенная (refined) *реконструкция* при помощи **features**, **matches**, **sfm_pairs**. В данном конфиге мы используем GT параметры известной для нас камеры, при этом не изменяя внутренние параметры камеры, а меняя только экстринсики камеры. Создание СNN features - 5 часов для 100 картинок (3 минуты в среднем на 1 картинку размером 2368х1952).

Ниже клетка работает примерно 10 минут для 100 картинок.

In [ ]:
if run_ref_single_with_params:
    conf = {
            "dense_features": {
                    "use_cache": True,
            },
             "KA": {
                    "dense_features": {'use_cache': True}, 
                    "split_in_subproblems": True,
                    "max_kps_per_problem": 1000,  
                },

            "BA": { 
                "apply": True,
                "optimizer": {
                      "refine_focal_length": False,  # whether to optimize the focal length
                      "refine_principal_point": False,  # whether to optimize the principal points
                      "refine_extra_params": False,  # whether to optimize distortion parameters
                      "refine_extrinsics": True,  # whether to optimize the camera poses
                }
            }
    }
    
    if has_cache == False:
        conf.update({
            "dense_features": {
                "use_cache": True,
                "sparse" : True,
                "dtype" : "half",
                "overwrite_cache": True,
                "load_cache_on_init": False,
                "patch_size": 8,
                "cache_format": "chunked"
            }
            
        })

    _cams = read_cameras_text(str(cameras_init))
    f, cx, cy, k = _cams[0].params

    opts = dict(camera_model='PINHOLE', camera_params=','.join(map(str, (f, cx, cy, k))))

    hloc_args = dict(camera_mode=pycolmap.CameraMode.SINGLE,
                    verbose=True,
                    image_options=opts)
    
    sfm = PixSfM(conf)
    
    if has_cache == False:
        refined1_single_params, sfm_outputs = sfm.reconstruction(exp1_dir_single_cam_with_params, 
                                              images, 
                                              sfm_pairs, 
                                              features, 
                                              matches, 
                                              image_list=references, 
                                              **hloc_args)
    else:
        refined1_single_params, sfm_outputs = sfm.reconstruction(exp1_dir_single_cam_with_params, 
                                              images, 
                                              sfm_pairs, 
                                              features, 
                                              matches, 
                                              image_list=references, 
                                              cache_path=cache_path,
                                              **hloc_args)
    
    print(refined1_single_params.summary())
    
    if not has_cache:    
        caches = root / f'pixel-perfect-sfm/outputs/caches/exp1/{object_name}'
        caches.mkdir(parents=True, exist_ok=True)
        cache_path = outputs / 's2dnet_featuremaps_sparse.h5'
        !cp -r $cache_path $caches  
    
    !mkdir -p $exp1_dir_single_cam_with_params/hloc/model_txt/

    !colmap model_converter \
        --input_path $exp1_dir_single_cam_with_params/hloc \
        --output_path $exp1_dir_single_cam_with_params/hloc/model_txt/ \
        --output_type TXT

# Visualization

Ниже сохранены все модели наших экспериментов. Мы их записали для того, чтобы Вы могли выбрать какую именно реконструкцию визуализировать при помощи plotly.

In [ ]:
refined_dict = {
    "raw_single": raw_single if run_raw_single else None,
    "raw_auto": raw_auto if run_raw_auto else None,
    "ref_single": refined1_single if run_ref_single else None,
    "ref_single_with_params": refined1_single_params if run_ref_single_with_params else None,
    "ref_auto": refined1_auto if run_ref_auto else None, 
}

Здесь мы сравниваем две реконструкции: сырую и улучшенную. Сравниваются положение камер, 3d pointclouds и local features на каком-то изображении.

In [ ]:
fig3d = init_figure()

args = dict(max_reproj_error=3.0, min_track_length=2, cs=1)
plot_reconstruction(fig3d, refined_dict['raw_single'], 
                    color='rgba(255, 0, 0, 0.5)', 
                    name="raw", **args)
plot_reconstruction(fig3d, refined_dict['ref_single_with_params'], 
                    color='rgba(0, 255, 0, 0.5)', 
                    name="refined", **args)
if show_visualization:
    fig3d.show()

In [ ]:
refined = refined_dict['ref_single_with_params']

img = refined.images[refined.reg_image_ids()[0]]
cam = refined.cameras[img.camera_id]

fig = init_image(images / img.name)    

plot_points2D(fig, [p2D.xy for p2D in img.points2D if p2D.has_point3D()])
plot_points2D(fig, cam.world_to_image(img.project(refined)), color='rgba(255, 0, 0, 0.5)')

if show_visualization:
    fig.show()